In [2]:
# import libraries
import pandas as pd

Importing datasets

In [3]:
# CBS OData API URL (JSON format)
url = "https://opendata.cbs.nl/ODataApi/OData/84487ENG/TypedDataSet"

# Read JSON and normalize to DataFrame
df = pd.json_normalize(pd.read_json(url)["value"])

# Preview
print(df.head())

HTTPError: HTTP Error 500: The given query is not allowed on this table. Please redefine your query so that it returns less than 10000 records.

Data Exploration - Games Dataframe